In [120]:
# key = input("이름을 입력하세요: ")
key = "정윤"

filename = "KakaoTalkChats.txt"
filename = "KakaoTalkChats (1).txt"
filename = "KakaoTalkChats (2).txt"
filename = "KakaoTalkChats (3).txt"
filename = "KakaoTalkChats (4).txt"

In [121]:
import re
from datetime import datetime
from collections import defaultdict

raw_chat = ""
with open(filename, 'r', encoding='utf-8') as f:
    raw_chat = f.read()

# 이모티콘, 사진 등의 비정형 메시지를 정리
def normalize_message(content):
    content = content.strip()
    if '이모티콘' in content:
        return '(이모티콘)'
    if '사진' in content:
        return '(사진)'

    # 감탄사, 말줄임표 치환
    content = re.sub(r'(!|ㄷ|ㅎ|ㅋ|ㅠ|ㅜ|\?|헐|하|헤|호|흐|허|와우|오오|진짜){2,}', r'\1\1', content)
    content = re.sub(r'\.{3,}', '...', content)
    # 링크 치환
    content = re.sub(r'http[s]?://\S+', '(링크)', content)
    # 계좌번호 (10~14자리)
    content = re.sub(r'\b\d{10,14}\b', '(계좌번호)', content)
    # 휴대폰/집전화번호
    content = re.sub(r'\b01[016789]-?\d{3,4}-?\d{4}\b', '(전화번호)', content)
    content = re.sub(r'\b(0\d{1,2}-?\d{3,4}-?\d{4})\b', '(전화번호)', content)
    # 카드번호
    content = re.sub(r'\b\d{4}-\d{4}-\d{4}-\d{4}|\d{16}\b', '(카드번호)', content)
    # 이메일 주소 치환
    content = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', '(이메일)', content)
    return content

# 라인별 파싱 및 정제
lines = raw_chat.strip().split('\n')
chat_entries = []

for line in lines:
    if "원을 보냈어요." in line or "원을 받았어요." in line or "원을 받으세요." in line or "받기 완료!" in line or "정산하기를 요청했어요." in line:
        continue
    match = re.match(r"(\d{4}년 \d{1,2}월 \d{1,2}일 (?:오전|오후) \d{1,2}:\d{2}), (.+?) : (.+)", line)

    if match:
        raw_time = match.group(1)
        sender = match.group(2).strip()
        if key in sender:
            sender = "나"
        else:
            sender = "상대방"
        message = normalize_message(match.group(3))
        
        # 시간 정규화
        try:
            dt = datetime.strptime(raw_time, "%Y년 %m월 %d일 오전 %I:%M")
        except:
            dt = datetime.strptime(raw_time, "%Y년 %m월 %d일 오후 %I:%M")
            dt = dt.replace(hour=dt.hour + 12 if dt.hour < 12 else dt.hour)
        chat_entries.append({
            'date': dt.strftime("%Y-%m-%d"),
            'time': dt.strftime("%H:%M"),
            'sender': sender,
            'message': message
        })


# 날짜별 > 시간별 > 발신자별 메시지 리스트
grouped_by_date = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

for entry in chat_entries:
    grouped_by_date[entry['date']][entry['time']][entry['sender']].append(entry['message'])

# 문자열 포맷 완성
formatted_lines = []

for date in sorted(grouped_by_date.keys()):
    formatted_lines.append(f"[{date}]")
    for time in sorted(grouped_by_date[date].keys()):
        for sender in grouped_by_date[date][time]:
            merged_msg = " | ".join(grouped_by_date[date][time][sender])
            formatted_lines.append(f"{time} - {sender}: {merged_msg}")
    formatted_lines.append("")

formatted_chat = "\n".join(formatted_lines).strip()
print(formatted_chat)

with open(f'processed/{filename}.txt', 'w', encoding='utf-8') as f:
    f.write(formatted_chat)

[2023-01-22]
11:25 - 상대방: 정윤아 | 이직준비함?
11:44 - 나: ? | 탈출각잡았다가 | 실패함
11:44 - 상대방: ㄹㅇ | 너회사돈얼마주냐 | 나요새 돈아 부족해서 나가고싶어짐 ㅋㅋ
11:57 - 나: ㅋㅋ쥐꼬리만큼 주지
11:58 - 상대방: ㄹㅇ
11:59 - 상대방: 어디로나직할거냐 | 어디로이직할거야?
12:34 - 나: ㅋㅋ네카라쿠베가고싶은데 | 갈수있을까?
12:35 - 나: 그래도 서울에 있는데로 | 생각중이긴한데 | 공부해야할게 넘많음 ㅠㅠ | 그치 | 그래도 서울에 일자리는 많으니까 아이티는 | 괜찮긴한데
12:35 - 상대방: 아이티구나 | ㄹㅇ.. | 서울에취업하는게 | 정말 메리트야.. | 이과라서 좀 어렵긴한데 | 그치
12:36 - 상대방: 전산쪽으로 | 노려봐
12:36 - 나: 전산? | 괜찮나
12:37 - 나: 전산쪽은 나도 잘아는편이아니라서 | ㅇㅇ알긴알지
12:37 - 상대방: 정처기? | 따면 | 할만하다햤었음 | 정보처리기사
12:38 - 나: 근데 리눅스마스터 이런것도 | 자격증따야 | 실무에 도움 된다던데
12:38 - 상대방: ㄹㅇ | 영어성적을 | 일단 챙겨보자
12:39 - 상대방: 안보는곳아 없드라 | 난 영어ㅜ개못해서.. | ㅋㅋ
12:39 - 나: ㅋㅋ | 토스 예전에 땃는데 | 레벨 6인가 | 근데 다시보면 | 안나올듯 ㅋㅋ
12:40 - 상대방: 난토익600도안나옴 ㅋㅋ | 토스 레벨5였나 | 삼성 기본 컷이.토스5라서 했던거샅으

[2023-03-31]
12:24 - 상대방: 정윤아 | 이직준비하니
12:25 - 나: ㅋㅋ | 반차아니고 | 패밀리데이라 | 일찍퇴근한거 | 도망각언제잡지
12:25 - 상대방: 걍나가고 | 준비를철저하게해볼까
12:26 - 나: 퇴사준비하냐 | ㅋㅋ
12:27 - 상대방: 아니퇴사하고싶은데
12:30 - 상대방: 어디로가야하오
12:34 - 나: 너 직업군은 갈데 정해져있음??
12:39 - 상대방: 그게머임? | 제약사업?
12:39 - 